# 晶体结构生成推理 Notebook - 迭代优化版

本notebook实现迭代优化的推理流程：

## 核心流程
1. **初始推理**：模型推理生成初始submission.csv
2. **质量评估**：使用RWP指标评估PXRD匹配质量
3. **迭代优化循环**：
   - 质量不好的样本进行后处理（能量优化→Rietveld精修）
   - 如果质量改善则更新submission.csv
   - 仍不满足要求的批量重新生成
4. **终止条件**：
   - 总运行时间超过5小时
   - 单个样本尝试次数超限
   - 所有样本满足质量要求

**注意**: 这个notebook设计为可直接在比赛环境运行

## 1. 导入必要的库和设置

In [ ]:
import json
import os
import sys
import time
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from pymatgen.core import Structure, Lattice, Composition, Element
import warnings
warnings.filterwarnings('ignore')

# 添加src目录到路径
sys.path.append('.')  # 添加根目录
sys.path.append('src')

# 导入必要的模块
from src.trainer import CrystalGenerationModule
from src.pxrd_simulator import PXRDSimulator
from src.normalizer import LatticeNormalizer

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA设备: {torch.cuda.get_device_name(0)}")

## 2. 配置参数

In [ ]:
# 数据路径配置
DATA_DIR = Path("data/A_sample")  # 比赛数据目录
COMPOSITION_FILE = DATA_DIR / "composition.json"
PATTERN_DIR = DATA_DIR / "pattern"

# 模型路径 - 使用实际的checkpoint路径
MODEL_PATH = "outputs/transformer_cfm_20250828_144134/checkpoints/last.ckpt"

# 输出文件（必须在根目录）
SUBMISSION_FILE = "submission.csv"

# 优化参数
RWP_THRESHOLD = 0.15  # RWP质量阈值，低于此值认为质量合格
MAX_TIME_HOURS = 5  # 最大运行时间（小时）
MAX_ATTEMPTS_PER_SAMPLE = 10  # 每个样本最大尝试次数
BATCH_SIZE = 32  # 批量重新生成的大小

# 记录开始时间
START_TIME = time.time()
MAX_RUNTIME = MAX_TIME_HOURS * 3600  # 转换为秒

print(f"配置参数：")
print(f"  模型路径: {MODEL_PATH}")
print(f"  模型存在: {os.path.exists(MODEL_PATH)}")
print(f"  RWP阈值: {RWP_THRESHOLD}")
print(f"  最大运行时间: {MAX_TIME_HOURS}小时")
print(f"  单样本最大尝试: {MAX_ATTEMPTS_PER_SAMPLE}次")
print(f"  开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 3. 数据加载函数

In [ ]:
def read_xy_file(file_path):
    """读取.xy格式的PXRD数据"""
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            parts = line.strip().split()
            if len(parts) >= 2:
                intensity = float(parts[1])
                data.append(intensity)
    return np.array(data, dtype=np.float32)

def parse_composition(comp_str):
    """解析组成字符串为原子类型和数量"""
    comp = Composition(comp_str)
    atom_list = []
    
    for element, count in comp.items():
        atomic_num = Element(element).Z
        atom_list.extend([atomic_num] * int(count))
    
    # 填充到60维
    atom_types = np.zeros(60, dtype=np.int32)
    atom_types[:len(atom_list)] = atom_list[:60]
    
    return len(atom_list), atom_types

def load_competition_data(data_dir):
    """加载比赛格式数据"""
    data_dir = Path(data_dir)
    
    # 读取composition
    with open(data_dir / "composition.json", 'r') as f:
        compositions = json.load(f)
    
    # 准备数据列表
    data_list = []
    
    for sample_id, comp_info in tqdm(compositions.items(), desc="加载数据"):
        # 获取组成信息
        comp_list = comp_info["composition"]
        niggli_comp = comp_list[0]
        primitive_comp = comp_list[1] if len(comp_list) > 1 else comp_list[0]
        
        # 解析原子信息
        num_atoms, atom_types = parse_composition(niggli_comp)
        
        # 读取PXRD数据
        pattern_file = data_dir / "pattern" / f"{sample_id}.xy"
        if pattern_file.exists():
            pxrd = read_xy_file(pattern_file)
            # 确保长度为11501
            if len(pxrd) < 11501:
                pxrd_full = np.zeros(11501, dtype=np.float32)
                pxrd_full[:len(pxrd)] = pxrd
                pxrd = pxrd_full
            elif len(pxrd) > 11501:
                pxrd = pxrd[:11501]
        else:
            pxrd = np.zeros(11501, dtype=np.float32)
        
        data_list.append({
            'id': sample_id,
            'niggli_comp': niggli_comp,
            'primitive_comp': primitive_comp,
            'atom_types': atom_types,
            'num_atoms': num_atoms,
            'pxrd': pxrd  # 观测的PXRD谱
        })
    
    return pd.DataFrame(data_list)

## 4. 加载数据

In [ ]:
# 加载比赛数据
df = load_competition_data(DATA_DIR)
print(f"\n加载了 {len(df)} 个样本")
print(f"数据列: {df.columns.tolist()}")
print(f"\n前5个样本:")
print(df[['id', 'niggli_comp', 'num_atoms']].head())

# 初始化样本状态追踪
sample_status = {
    sample_id: {
        'attempts': 0,
        'best_rwp': float('inf'),
        'best_structure': None,
        'satisfied': False
    }
    for sample_id in df['id']
}

print(f"\n初始化了 {len(sample_status)} 个样本的状态追踪")

## 5. 模型和推理函数

In [ ]:
# 加载模型和初始化工具
def load_model(model_path):
    """
    加载训练好的模型
    
    Args:
        model_path: checkpoint文件路径
        
    Returns:
        加载好的Lightning模块
    """
    print(f"正在加载模型: {model_path}")
    
    # 从checkpoint加载模型
    model = CrystalGenerationModule.load_from_checkpoint(
        model_path,
        map_location='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    # 设置为评估模式
    model.eval()
    
    # 移动到正确的设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    print(f"模型加载成功，设备: {device}")
    return model

def prepare_batch_for_model(sample_df, device):
    """
    准备模型输入批次
    
    Args:
        sample_df: 包含样本数据的DataFrame
        device: 计算设备
        
    Returns:
        dict: 模型输入批次
    """
    batch = {
        'comp': torch.tensor(np.stack(sample_df['atom_types'].values), dtype=torch.float32).to(device),
        'pxrd': torch.tensor(np.stack(sample_df['pxrd'].values), dtype=torch.float32).to(device),
        'num_atoms': torch.tensor(sample_df['num_atoms'].values, dtype=torch.long).to(device),
    }
    return batch

def generate_crystal_structure(sample, model, lattice_normalizer, pxrd_simulator):
    """
    使用模型生成晶体结构
    
    Args:
        sample: 包含pxrd、atom_types、num_atoms等信息的样本
        model: 训练好的模型
        lattice_normalizer: 晶格归一化器
        pxrd_simulator: PXRD仿真器
    
    Returns:
        Structure对象
    """
    device = next(model.parameters()).device
    
    # 准备单个样本的批次
    batch = {
        'comp': torch.tensor(sample['atom_types'], dtype=torch.float32).unsqueeze(0).to(device),
        'pxrd': torch.tensor(sample['pxrd'], dtype=torch.float32).unsqueeze(0).to(device),
        'num_atoms': torch.tensor([sample['num_atoms']], dtype=torch.long).to(device),
    }
    
    # 模型推理
    with torch.no_grad():
        # 使用flow的sample方法生成
        generated = model.flow.sample(batch)  # [1, 63, 3]
        
    # 转换到CPU并提取数据
    generated = generated.cpu().numpy()[0]  # [63, 3]
    
    # 分离晶格和分数坐标
    lattice_vectors = generated[:3, :]  # [3, 3] 归一化的晶格向量
    frac_coords = generated[3:3+sample['num_atoms'], :]  # [num_atoms, 3]
    
    # 反归一化晶格参数
    lattice_matrix = lattice_normalizer.denormalize_lattice(
        torch.tensor(lattice_vectors, dtype=torch.float32).unsqueeze(0)
    ).numpy()[0]
    
    # 确保分数坐标在[0,1]范围内
    frac_coords = np.mod(frac_coords, 1.0)
    
    # 获取元素列表
    species = []
    for i in range(sample['num_atoms']):
        atomic_num = int(sample['atom_types'][i])
        if atomic_num > 0:
            species.append(Element.from_Z(atomic_num))
    
    # 创建Structure对象
    try:
        lattice = Lattice(lattice_matrix)
        structure = Structure(
            lattice=lattice,
            species=species,
            coords=frac_coords,
            coords_are_cartesian=False
        )
    except Exception as e:
        print(f"创建Structure失败: {e}")
        # 回退到随机生成
        return generate_random_structure(sample)
    
    return structure

def generate_random_structure(sample):
    """
    生成随机晶体结构（备用方案）
    
    Args:
        sample: 样本数据
        
    Returns:
        Structure对象
    """
    num_atoms = sample['num_atoms']
    
    # 随机晶格参数
    a = np.random.uniform(3, 10)
    b = np.random.uniform(3, 10)
    c = np.random.uniform(3, 10)
    alpha = np.random.uniform(60, 120)
    beta = np.random.uniform(60, 120)
    gamma = np.random.uniform(60, 120)
    
    lattice = Lattice.from_parameters(a, b, c, alpha, beta, gamma)
    frac_coords = np.random.rand(num_atoms, 3)
    
    species = []
    for i in range(num_atoms):
        atomic_num = int(sample['atom_types'][i])
        if atomic_num > 0:
            species.append(Element.from_Z(atomic_num))
    
    return Structure(
        lattice=lattice,
        species=species,
        coords=frac_coords,
        coords_are_cartesian=False
    )

# 加载模型和初始化工具
try:
    model = load_model(MODEL_PATH)
    lattice_normalizer = LatticeNormalizer()
    pxrd_simulator = PXRDSimulator()
    print("✅ 模型和工具初始化成功")
except Exception as e:
    print(f"⚠️ 模型加载失败: {e}")
    print("将使用随机生成作为备用方案")
    model = None
    lattice_normalizer = None
    pxrd_simulator = None

## 6. PXRD计算和质量评估

In [ ]:
def calculate_pxrd(structure, pxrd_simulator=None):
    """
    计算晶体结构的PXRD谱
    
    Args:
        structure: pymatgen Structure对象
        pxrd_simulator: PXRD仿真器实例
    
    Returns:
        np.array: 11501维的PXRD强度数组
    """
    if pxrd_simulator is None:
        # 如果没有提供simulator，创建一个新的
        from src.PXRDSimulator import PXRDSimulator
        pxrd_simulator = PXRDSimulator()
    
    try:
        # 使用PXRDSimulator计算PXRD
        pxrd_calc = pxrd_simulator.get_pattern(structure)
        
        # 确保返回11501维
        if len(pxrd_calc) < 11501:
            pxrd_full = np.zeros(11501, dtype=np.float32)
            pxrd_full[:len(pxrd_calc)] = pxrd_calc
            pxrd_calc = pxrd_full
        elif len(pxrd_calc) > 11501:
            pxrd_calc = pxrd_calc[:11501]
            
        return pxrd_calc
        
    except Exception as e:
        print(f"PXRD计算失败: {e}")
        # 返回随机PXRD作为备用
        pxrd_calc = np.random.rand(11501) * 100
        pxrd_calc[pxrd_calc < 10] = 0
        return pxrd_calc

def evaluate_structure_quality(structure, observed_pxrd, pxrd_simulator=None):
    """
    评估生成结构的质量
    
    Args:
        structure: 生成的Structure对象
        observed_pxrd: 观测的PXRD谱
        pxrd_simulator: PXRD仿真器实例
    
    Returns:
        float: RWP值（越小越好）
    """
    # 计算生成结构的PXRD
    calculated_pxrd = calculate_pxrd(structure, pxrd_simulator)
    
    # 简单的RWP计算（如果没有专门的metrics模块）
    try:
        from src.metrics import rwp
        rwp_value = rwp(calculated_pxrd, observed_pxrd)
    except ImportError:
        # 备用RWP计算
        diff = calculated_pxrd - observed_pxrd
        weighted_diff = diff * np.sqrt(np.maximum(observed_pxrd, 1e-10))
        rwp_value = np.sqrt(np.sum(weighted_diff**2) / np.sum(observed_pxrd**2 + 1e-10))
    
    return rwp_value

## 7. 后处理函数

In [ ]:
def energy_optimization(structure):
    """
    能量优化
    
    Args:
        structure: 待优化的Structure对象
    
    Returns:
        Structure: 优化后的结构
    """
    # TODO: 实现能量优化
    # 可以使用GULP、VASP、或机器学习势函数等
    
    # 占位：稍微调整晶格参数模拟优化
    new_lattice = structure.lattice.matrix * np.random.uniform(0.98, 1.02)
    optimized = Structure(
        lattice=Lattice(new_lattice),
        species=structure.species,
        coords=structure.frac_coords,
        coords_are_cartesian=False
    )
    
    return optimized

def rietveld_refinement(structure, observed_pxrd):
    """
    Rietveld精修
    
    Args:
        structure: 待精修的Structure对象
        observed_pxrd: 观测的PXRD谱
    
    Returns:
        tuple: (精修后的Structure, 是否需要精修)
    """
    # 判断是否需要精修
    current_rwp = evaluate_structure_quality(structure, observed_pxrd)
    needs_refinement = current_rwp > RWP_THRESHOLD * 1.5  # 如果RWP较高则需要精修
    
    if not needs_refinement:
        return structure, False
    
    # TODO: 实现Rietveld精修
    # 可以使用GSAS-II、TOPAS、FullProf等
    
    # 占位：稍微调整原子位置模拟精修
    new_coords = structure.frac_coords + np.random.randn(*structure.frac_coords.shape) * 0.01
    new_coords = np.clip(new_coords, 0, 1)  # 确保在[0,1]范围内
    
    refined = Structure(
        lattice=structure.lattice,
        species=structure.species,
        coords=new_coords,
        coords_are_cartesian=False
    )
    
    return refined, True

def post_process_structure(structure, observed_pxrd):
    """
    完整的后处理流程
    
    Args:
        structure: 待处理的Structure对象
        observed_pxrd: 观测的PXRD谱
    
    Returns:
        tuple: (处理后的Structure, 最终RWP值)
    """
    # 1. 能量优化
    optimized = energy_optimization(structure)
    rwp_after_opt = evaluate_structure_quality(optimized, observed_pxrd)
    
    # 2. Rietveld精修（如果需要）
    refined, was_refined = rietveld_refinement(optimized, observed_pxrd)
    
    if was_refined:
        rwp_after_refine = evaluate_structure_quality(refined, observed_pxrd)
        return refined, rwp_after_refine
    else:
        return optimized, rwp_after_opt

## 8. 终止条件检查

In [ ]:
def check_termination_conditions(sample_status):
    """
    检查是否满足终止条件
    
    终止条件：
    1. 运行时间超过5小时
    2. 所有样本都满足质量要求或达到最大尝试次数
    
    Returns:
        tuple: (是否终止, 终止原因)
    """
    # 检查运行时间
    elapsed_time = time.time() - START_TIME
    if elapsed_time > MAX_RUNTIME:
        return True, f"达到最大运行时间 {MAX_TIME_HOURS} 小时"
    
    # 检查所有样本状态
    all_done = all(
        status['satisfied'] or status['attempts'] >= MAX_ATTEMPTS_PER_SAMPLE
        for status in sample_status.values()
    )
    
    if all_done:
        satisfied_count = sum(1 for s in sample_status.values() if s['satisfied'])
        return True, f"所有样本处理完成（{satisfied_count}/{len(sample_status)}满足要求）"
    
    return False, None

def get_samples_to_regenerate(sample_status, batch_size=32):
    """
    获取需要重新生成的样本
    
    Args:
        sample_status: 样本状态字典
        batch_size: 批次大小
    
    Returns:
        list: 需要重新生成的样本ID列表
    """
    # 找出未满足要求且未超过尝试次数的样本
    candidates = [
        sample_id for sample_id, status in sample_status.items()
        if not status['satisfied'] and status['attempts'] < MAX_ATTEMPTS_PER_SAMPLE
    ]
    
    # 按RWP值排序，优先处理质量最差的
    candidates.sort(key=lambda x: sample_status[x]['best_rwp'], reverse=True)
    
    return candidates[:batch_size]

## 12. 初始推理（带实时保存）

In [ ]:
print("="*60)
print("阶段1：初始推理")
print("="*60)

initial_predictions = {}

for idx, row in tqdm(df.iterrows(), total=len(df), desc="初始推理"):
    sample_id = row['id']
    
    # 生成初始结构
    if model is not None:
        structure = generate_crystal_structure(row, model, lattice_normalizer, pxrd_simulator)
    else:
        structure = generate_random_structure(row)
    
    # 评估质量
    rwp_value = evaluate_structure_quality(structure, row['pxrd'], pxrd_simulator)
    
    # 更新状态
    sample_status[sample_id]['attempts'] = 1
    sample_status[sample_id]['best_rwp'] = rwp_value
    sample_status[sample_id]['best_structure'] = structure
    sample_status[sample_id]['satisfied'] = rwp_value < RWP_THRESHOLD
    
    initial_predictions[sample_id] = structure

# 统计初始结果
satisfied_count = sum(1 for s in sample_status.values() if s['satisfied'])
avg_rwp = np.mean([s['best_rwp'] for s in sample_status.values()])

print(f"\n初始推理结果:")
print(f"  满足要求: {satisfied_count}/{len(sample_status)} ({satisfied_count/len(sample_status)*100:.1f}%)")
print(f"  平均RWP: {avg_rwp:.4f}")
print(f"  RWP阈值: {RWP_THRESHOLD}")

# 立即保存初始推理结果到submission.csv
submission_df = update_submission_incrementally(sample_status, DATA_DIR, SUBMISSION_FILE)
log_submission_update(0, sample_status, SUBMISSION_FILE)

## 13. 迭代优化循环（带实时保存）

In [ ]:
print("\n" + "="*60)
print("阶段2：迭代优化")
print("="*60)

iteration = 0
while True:
    iteration += 1
    
    # 检查终止条件
    should_terminate, reason = check_termination_conditions(sample_status)
    if should_terminate:
        print(f"\n终止优化: {reason}")
        break
    
    print(f"\n--- 迭代 {iteration} ---")
    elapsed = time.time() - START_TIME
    print(f"已运行: {elapsed/3600:.2f}小时")
    
    # 获取需要优化的样本
    samples_to_process = get_samples_to_regenerate(sample_status, BATCH_SIZE)
    
    if not samples_to_process:
        print("没有需要处理的样本")
        break
    
    print(f"处理 {len(samples_to_process)} 个样本")
    
    # 标记是否有改进
    has_improvement = False
    
    # 处理每个样本
    for sample_id in tqdm(samples_to_process, desc=f"迭代{iteration}"):
        row = df[df['id'] == sample_id].iloc[0]
        current_best = sample_status[sample_id]['best_structure']
        current_rwp = sample_status[sample_id]['best_rwp']
        initial_rwp = current_rwp  # 记录改进前的RWP
        
        # 策略1：后处理当前最佳结构
        if sample_status[sample_id]['attempts'] < MAX_ATTEMPTS_PER_SAMPLE // 2:
            processed_structure, processed_rwp = post_process_structure(
                current_best, row['pxrd']
            )
            
            if processed_rwp < current_rwp:
                sample_status[sample_id]['best_structure'] = processed_structure
                sample_status[sample_id]['best_rwp'] = processed_rwp
                sample_status[sample_id]['satisfied'] = processed_rwp < RWP_THRESHOLD
                current_rwp = processed_rwp
                has_improvement = True
        
        # 策略2：如果质量仍不满足，重新生成
        if not sample_status[sample_id]['satisfied']:
            if model is not None:
                new_structure = generate_crystal_structure(row, model, lattice_normalizer, pxrd_simulator)
            else:
                new_structure = generate_random_structure(row)
                
            new_rwp = evaluate_structure_quality(new_structure, row['pxrd'], pxrd_simulator)
            
            # 后处理新生成的结构
            processed_new, processed_new_rwp = post_process_structure(
                new_structure, row['pxrd']
            )
            
            # 保留最佳结果
            if processed_new_rwp < sample_status[sample_id]['best_rwp']:
                sample_status[sample_id]['best_structure'] = processed_new
                sample_status[sample_id]['best_rwp'] = processed_new_rwp
                sample_status[sample_id]['satisfied'] = processed_new_rwp < RWP_THRESHOLD
                has_improvement = True
        
        # 更新尝试次数
        sample_status[sample_id]['attempts'] += 1
    
    # 统计当前状态
    satisfied_count = sum(1 for s in sample_status.values() if s['satisfied'])
    avg_rwp = np.mean([s['best_rwp'] for s in sample_status.values()])
    
    print(f"\n迭代{iteration}结果:")
    print(f"  满足要求: {satisfied_count}/{len(sample_status)} ({satisfied_count/len(sample_status)*100:.1f}%)")
    print(f"  平均RWP: {avg_rwp:.4f}")
    
    # 每次迭代后都更新submission.csv（无论是否有改进）
    submission_df = update_submission_incrementally(sample_status, DATA_DIR, SUBMISSION_FILE)
    log_submission_update(iteration, sample_status, SUBMISSION_FILE)
    
    # 如果有改进，输出改进信息
    if has_improvement:
        print("  ✨ 本轮有样本得到改进")
    
    # 限制迭代次数（额外保护）
    if iteration > 100:
        print("达到最大迭代次数")
        break

print("\n" + "="*60)
print("迭代优化完成")
print("="*60)

## 11. 增量更新submission.csv函数

In [ ]:
def update_submission_incrementally(sample_status, data_dir, output_file="submission.csv"):
    """
    增量更新submission.csv文件
    每次调用时重新生成整个文件，确保包含最新的所有结果
    
    Args:
        sample_status: 样本状态字典，包含每个样本的最佳结构
        data_dir: 数据目录路径
        output_file: 输出文件名
    
    Returns:
        pd.DataFrame: submission数据框
    """
    # 获取ID前缀（A或B）
    with open(data_dir / "composition.json", 'r') as f:
        composition_dict = json.load(f)
    prefix = next(iter(composition_dict))[0]  # 获取第一个ID的首字母
    
    # 准备submission数据
    rows = []
    
    for sample_id, status in sample_status.items():
        try:
            structure = status['best_structure']
            
            if structure is not None:
                # 转换为CIF格式
                cif_str = structure.to(fmt="cif")
            else:
                # 如果还没有结构，创建占位CIF
                cif_str = f"data_{sample_id}\n_cell_length_a 5.0\n_cell_length_b 5.0\n_cell_length_c 5.0\n_cell_angle_alpha 90\n_cell_angle_beta 90\n_cell_angle_gamma 90\n"
            
            rows.append({
                'ID': sample_id,
                'cif': cif_str
            })
        except Exception as e:
            # 出错时创建占位CIF
            min_cif = f"data_{sample_id}\n_cell_length_a 5.0\n_cell_length_b 5.0\n_cell_length_c 5.0\n_cell_angle_alpha 90\n_cell_angle_beta 90\n_cell_angle_gamma 90\n"
            rows.append({
                'ID': sample_id,
                'cif': min_cif
            })
    
    # 创建DataFrame
    submission_df = pd.DataFrame(rows)
    
    # 保存为CSV（覆盖原文件）
    submission_df.to_csv(output_file, index=False)
    
    return submission_df

def log_submission_update(iteration, sample_status, submission_file="submission.csv"):
    """
    记录submission更新信息
    
    Args:
        iteration: 当前迭代轮次（0表示初始推理）
        sample_status: 样本状态字典
        submission_file: submission文件路径
    """
    satisfied_count = sum(1 for s in sample_status.values() if s['satisfied'])
    total_count = len(sample_status)
    
    if iteration == 0:
        print(f"\n📝 初始submission.csv已生成")
    else:
        print(f"\n📝 submission.csv已更新 (迭代{iteration})")
    
    print(f"   满足要求: {satisfied_count}/{total_count} ({satisfied_count/total_count*100:.1f}%)")
    
    if os.path.exists(submission_file):
        file_size = os.path.getsize(submission_file) / 1024
        print(f"   文件大小: {file_size:.2f} KB")

## 14. 最终统计和验证

In [ ]:
# 最终统计
print("\n" + "="*60)
print("最终统计")
print("="*60)

# 计算各项统计指标
satisfied_samples = [s for s in sample_status.values() if s['satisfied']]
unsatisfied_samples = [s for s in sample_status.values() if not s['satisfied']]

print(f"\n质量统计:")
print(f"  满足RWP<{RWP_THRESHOLD}: {len(satisfied_samples)}/{len(sample_status)} ({len(satisfied_samples)/len(sample_status)*100:.1f}%)")
print(f"  未满足要求: {len(unsatisfied_samples)}")

if satisfied_samples:
    satisfied_rwps = [s['best_rwp'] for s in satisfied_samples]
    print(f"\n满足要求样本的RWP:")
    print(f"  最小: {np.min(satisfied_rwps):.4f}")
    print(f"  最大: {np.max(satisfied_rwps):.4f}")
    print(f"  平均: {np.mean(satisfied_rwps):.4f}")

if unsatisfied_samples:
    unsatisfied_rwps = [s['best_rwp'] for s in unsatisfied_samples]
    print(f"\n未满足要求样本的RWP:")
    print(f"  最小: {np.min(unsatisfied_rwps):.4f}")
    print(f"  最大: {np.max(unsatisfied_rwps):.4f}")
    print(f"  平均: {np.mean(unsatisfied_rwps):.4f}")

# 尝试次数统计
attempts_list = [s['attempts'] for s in sample_status.values()]
print(f"\n尝试次数统计:")
print(f"  最少: {np.min(attempts_list)}")
print(f"  最多: {np.max(attempts_list)}")
print(f"  平均: {np.mean(attempts_list):.1f}")
print(f"  达到上限({MAX_ATTEMPTS_PER_SAMPLE}次): {sum(1 for a in attempts_list if a >= MAX_ATTEMPTS_PER_SAMPLE)}")

# 运行时间
total_time = time.time() - START_TIME
print(f"\n总运行时间: {total_time/3600:.2f}小时")

# 验证最终的提交文件
print(f"\n验证最终submission文件:")
if os.path.exists(SUBMISSION_FILE):
    # 重新读取文件以验证
    final_submission = pd.read_csv(SUBMISSION_FILE)
    print(f"  文件名: {SUBMISSION_FILE}")
    print(f"  文件大小: {os.path.getsize(SUBMISSION_FILE) / 1024:.2f} KB")
    print(f"  样本数: {len(final_submission)}")
    print(f"  列名: {final_submission.columns.tolist()}")
    
    # 检查是否有缺失值
    missing = final_submission.isnull().sum()
    if missing.any():
        print(f"\n⚠️ 警告：发现缺失值！")
        print(missing[missing > 0])
    else:
        print(f"  ✅ 没有缺失值")
    
    # 检查ID是否完整
    expected_ids = set(sample_status.keys())
    actual_ids = set(final_submission['ID'].values)
    if expected_ids == actual_ids:
        print(f"  ✅ 所有样本ID都已包含")
    else:
        missing_ids = expected_ids - actual_ids
        extra_ids = actual_ids - expected_ids
        if missing_ids:
            print(f"  ⚠️ 缺少ID: {missing_ids}")
        if extra_ids:
            print(f"  ⚠️ 多余ID: {extra_ids}")
else:
    print(f"  ❌ 文件不存在: {SUBMISSION_FILE}")

print("\n" + "="*60)
print("✅ 推理完成！submission.csv已在整个过程中实时更新")
print("="*60)

## 总结

本notebook实现了迭代优化的推理流程，并**实时更新submission.csv**：

### 核心流程
1. ✅ **初始推理**：模型推理生成初始结构，立即保存submission.csv
2. ✅ **质量评估**：使用RWP指标评估PXRD匹配质量
3. ✅ **迭代优化**：
   - 能量优化 + Rietveld精修
   - **每轮迭代后立即更新submission.csv**
   - 批量重新生成不满足要求的样本
4. ✅ **终止条件**：
   - 运行时间限制（5小时）
   - 单样本尝试次数限制
   - 全部满足要求

### 关键改进
- 🔄 **增量更新机制**：每次推理/优化后立即覆盖submission.csv
- 📊 **实时进度反馈**：评测脚本可以随时读取最新结果
- 🛡️ **断点续传支持**：即使中途中断，已有结果也保存在submission.csv中
- 📝 **更新日志**：每次更新都记录状态信息（满足率、文件大小等）

### 待实现部分
- ⏳ 实际的模型推理
- ⏳ PXRD计算（调用PXRDSimulator）
- ⏳ 能量优化（GULP等）
- ⏳ Rietveld精修（GSAS-II等）

### 输出
- ✅ 符合比赛要求的submission.csv（CIF格式）
- ✅ **实时更新**：每轮推理后立即保存，评测脚本可及时读取
- ✅ 详细的优化过程记录和统计